<a href="https://colab.research.google.com/github/hate-speech-classification/abusive-level-classification/blob/main/abusive_speech_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
"""
Init
"""
import logging
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
nltk.download('stopwords')
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup
%matplotlib inline
# Import necessary modules
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import LeavePOut
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import StratifiedKFold

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [24]:
"""
Create new column labels
"""
def categorise(row):
  if row['Annotator1_is_abuse.1'] == '1':
    return 'Not abusive'
  elif row['Annotator1_is_abuse.0'] == '1':
    return "Ambiguous"
  elif row['Annotator1_is_abuse.-1'] == '1':
    return "Mildly abusive"
  elif row['Annotator1_is_abuse.-2'] == '1':
    return "Strongly abusive"
  elif row['Annotator1_is_abuse.-3'] == '1':
    return "Very strongly abusive"


In [25]:
"""
Clean text (user column)
"""
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    return text


In [61]:
"""
MAIN MODEL IMPLEMENTATION
-------------------------------------------
NLP ALGO: TFIDF
MODEL: Linear Support Vector Machine
VALIDATION METHOD: kFold
"""
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
def predict_abusive_level(userInput):
  url_train_data = 'https://raw.githubusercontent.com/amandacurry/convabuse/main/2_splits/ConvAbuseEMNLPtrain.csv'
  df = pd.read_csv(url_train_data)
  my_tags = ['Not abusive', 'Ambiguous', 'Midly abusive', 'Strongly abusive', 'Very strongly abusive']
  df['abusive_level'] = df.apply(lambda row: categorise(row), axis=1) 
  df = df[pd.notnull(df['abusive_level'])]
  df['user'] = df['user'].apply(clean_text)
  data = df[['user','abusive_level']]
  X = data.user
  Y = data.abusive_level
  kfold = KFold(n_splits=10, random_state=100, shuffle=True)
  sgd = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)),
               ])
  ml = sgd.fit(X, Y)
  print(ml.predict([userInput]))

In [66]:
predict_abusive_level("I hope you die")


['Very strongly abusive']
